In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as plt

In [2]:
from NRI.experiments.config import ExperimentConfig


experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=18,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_18_baseline',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=False
)

In [3]:
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset

dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
train_files, valid_files, test_files = split_dataset(dataset_files, 0.7, 0.2)

load datasets

In [4]:
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0
)

train_set = SignalizedIntersectionDatasetForNRI(dataset_config)
train_set.load_records(dataset_path, train_files, verbose=True)

valid_set = SignalizedIntersectionDatasetForNRI(dataset_config)
valid_set.load_records(dataset_path, valid_files, verbose=True)


load_records: 100%|██████████| 4/4 [00:41<00:00, 10.29s/it]


In [5]:
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.main import train

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

if experiment_config.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train(
    model,
    train_set=train_set,
    valid_set=valid_set,
    config=experiment_config,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler
)

[train] epoch 0:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Overall Loss: 23.12710672201112
Encoder KL Loss: 0.020016891099404296
Decoder NLL Loss: 21.974801623544014
Signal Cross-Entropy Loss: 1.1322880931371868


[valid] epoch 0:   0%|          | 0/48 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.7695481777191162
Final Displacement Error: 6.4911322593688965
Average Displacement Error: 3.0889740685621896


[train] epoch 1:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

...

Signal Prediction Accuracy: 0.8257908821105957
Final Displacement Error: 4.124460392942031
Average Displacement Error: 1.769045827910304


[train] epoch 19:   0%|          | 0/172 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: 

Overall Loss: -5.776605830636134
Encoder KL Loss: 0.08771735997220807
Decoder NLL Loss: -6.467666645382725
Signal Cross-Entropy Loss: 0.603343469458957


[valid] epoch 19:   0%|          | 0/48 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Signal Prediction Accuracy: 0.8241451978683472
Final Displacement Error: 4.068690974265337
Average Displacement Error: 1.7385124439994495


In [1]:
import os
import torch
from torch.utils.data import DataLoader

from NRI.dataset import SignalizedIntersectionDatasetForNRI
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.config import ExperimentConfig
from NRI.experiments.test import generate_result
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_18_baseline',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=False
)

# Load the dataset
dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
_, _, test_files = split_dataset(dataset_files, 0.7, 0.2)

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0,
    include_incomplete_trajectories=False
)

test_set = SignalizedIntersectionDatasetForNRI(dataset_config)
test_set.load_records(dataset_path, test_files, verbose=True)

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

# Load the saved model and configuration
checkpoint = torch.load('../checkpoints/dnri_18_baseline_best.pt')
model.load_state_dict(checkpoint['params'])
test_stats = generate_result(model, test_set, experiment_config)

# Report the results
test_stats.report()

[test] generating result:   0%|          | 0/34 [00:00<?, ?it/s]c:\Users\chinp\source\AnythingVisionaries\venv\Lib\site-packages\torch\nested\__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ..\aten\src\ATen\NestedTensorImpl.cpp:180.)
  return _nested.nested_tensor(
[test] generating result: 100%|██████████| 34/34 [00:06<00:00,  5.49it/s]

=== Overall Result ===
Signal Prediction Accuracy: 0.7773651480674744
Overall Final Displacement Error: 2.0950443744659424
Overall Average Displacement Error: 0.86806720495224
=== Displacement Error of Class `pedestrian` ===
FDE: 4.165600869019444
ADE: 2.2922888979774254
=== Displacement Error of Class `car` ===
FDE: 2.2438755868319338
ADE: 0.6204790487683736
=== Displacement Error of Class `truck` ===
FDE: 20.68140629799135
ADE: 5.773994162077865
=== Displacement Error of Class `bus` ===
FDE: 0.1488626353264396
ADE: 0.07469738022727695
=== Displacement Error of Class `motorcycle` ===
FDE: 2.3508817310053747
ADE: 0.8774539249190506
=== Displacement Error of Class `tricycle` ===
FDE: 0.8040027418429235
ADE: 0.3618348764101934
=== Displacement Error of Class `bicycle` ===
FDE: 3.7436919240275945
ADE: 1.668580631065446
=== Overall Result ===
Signal Prediction Accuracy: 0.7773651480674744
Overall Final Displacement Error: 2.0950443744659424
Overall Average Displacement Error: 0.86806720495